In [1]:
import os
os.getcwd()
os.chdir("drive/My Drive/STAT212/DeepZip_code/src")

In [2]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn.utils.prune as prune
import torch.quantization
import copy 
import os
import zipfile
import tempfile
import shutil

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
torch.manual_seed(1)

In [3]:
file_path="../data/processed_files/text8.npy"
batch_size=128
time_steps= 64

In [4]:
def strided_app(a, L, S):  # Window len = L, Stride len/stepsize = S
        nrows = ((a.size - L) // S) + 1
        n = a.strides[0]
        return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)

In [9]:
# load the preprocessed data
series = np.load(file_path)
series = series.reshape(-1, 1)
series=series[0:100000]

onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit(series)
series = series.reshape(-1)

data = strided_app(series, time_steps+1, 1)
l = int(len(data)/batch_size) * batch_size

data = data[:l] 
X = data[:, :-1]
Y = data[:, -1:].reshape([-1,])
Y_hot = onehot_encoder.transform(data[:, -1:])

In [10]:
# Hyper Parameters
num_epochs=10           
input_size = 1   
hidden_size = 64
num_layers = 1
num_classes = Y_hot.shape[1]
lr = 0.01   


# Define LSTM model
class simpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(simpleLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True).to(device)
        self.fc = nn.Linear(hidden_size, num_classes).to(device)

    def forward(self, x):
        # initialize
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # forward propagate lstm
        out, (h_n, h_c) = self.lstm(x, (h0, c0))

        # output
        out =self.fc(out[:, -1, :])
        return out

model = simpleLSTM(input_size, hidden_size, num_layers, num_classes)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
# load training data
train_data = TensorDataset(torch.Tensor(X),torch.Tensor(Y).long())
train_loader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True) 

# train
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        x = x.reshape(-1, time_steps, input_size).to(device)
        y = y.to(device)

        # forward pass
        outputs = model(x)
        loss = criterion(outputs, y)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [8]:
# test accuary
test_data = TensorDataset(torch.Tensor(X),torch.Tensor(Y).long())
test_loader = DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True) 
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        x = x.reshape(-1, time_steps, input_size).to(device)
        y = y.to(device)
        outputs = model(x)
        prob=F.softmax(outputs)
        _, predicted = torch.max(prob, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    print('Test Accuracy of the model on the 10000 test x: {} %'.format(100 * correct / total))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Test Accuracy of the model on the 10000 test x: 46.03461593878074 %


In [9]:
torch.save(model.state_dict(), "../data/trained_models/text8/temptory/lstm_weights")

In [10]:
# copy a model0 for pruning
model0=copy.deepcopy(model)

In [11]:
for name, module in model0.named_modules():
    # prune 20% of connections in all lstm layers
    if isinstance(module, torch.nn.LSTM):
        prune.l1_unstructured(module, name='weight_hh_l0', amount=0.2)
        prune.l1_unstructured(module, name='weight_ih_l0', amount=0.2)
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

for name, module in model0.named_modules():
    # prune 20% of connections in all lstm layers
    if isinstance(module, torch.nn.LSTM):
        prune.remove(module, 'weight_hh_l0')
        prune.remove(module, 'weight_ih_l0')
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        prune.remove(module, 'weight')

In [12]:
torch.save(model0.state_dict(), "../data/trained_models/text8/temptory/lstm_pruned_weights")

In [13]:
# quantize the unpruned model
quantized_model = torch.quantization.quantize_dynamic(
    model.to('cpu'), {nn.LSTM, nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

# quantize the pruned model
quantized_model0 = torch.quantization.quantize_dynamic(
    model0.to('cpu'), {nn.LSTM, nn.Linear}, dtype=torch.qint8
)
print(quantized_model0)

simpleLSTM(
  (lstm): DynamicQuantizedLSTM(1, 64, batch_first=True)
  (fc): DynamicQuantizedLinear(in_features=64, out_features=27, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)
simpleLSTM(
  (lstm): DynamicQuantizedLSTM(1, 64, batch_first=True)
  (fc): DynamicQuantizedLinear(in_features=64, out_features=27, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)


In [14]:
torch.save(quantized_model.state_dict(), "../data/trained_models/text8/temptory/lstm_quantization_weights")
torch.save(quantized_model0.state_dict(), "../data/trained_models/text8/temptory/lstm_quantization_pruned_weights")

In [15]:
# define a gzip function
def get_gzipped_model_size(file,path):
  # Returns size of gzipped model, in bytes.
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  shutil.copy(zipped_file, path)
  return os.path.getsize(zipped_file)

In [16]:
os.chdir("../data/trained_models/text8/temptory")
file_path="lstm_weights"
save_path="../lstm_weights.zip"
print("Size of gzipped baseline model: %.2f bytes" % (get_gzipped_model_size(file_path,save_path)))

file_path="lstm_pruned_weights"
save_path="../lstm_pruned_weights.zip"
print("Size of gzipped pruned model: %.2f bytes" % (get_gzipped_model_size(file_path,save_path)))

file_path="lstm_quantization_weights"
save_path="../lstm_quantization_weights.zip"
print("Size of gzipped quantized baseline model: %.2f bytes" % (get_gzipped_model_size(file_path,save_path)))

file_path="lstm_quantization_pruned_weights"
save_path="../lstm_quantization_pruned_weights.zip"
print("Size of gzipped quantized and pruned model: %.2f bytes" % (get_gzipped_model_size(file_path,save_path)))

Size of gzipped baseline model: 71837.00 bytes
Size of gzipped pruned model: 61241.00 bytes
Size of gzipped quantized baseline model: 18921.00 bytes
Size of gzipped quantized and pruned model: 17742.00 bytes
